In [4]:
import pandas as pd
data = pd.read_excel('data.xls')
data.head()

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,病程阶段,TNM分期,转移部位,确诊后几年发现转移
0,0.056,0.460,0.281,0.352,0.119,0.350,S4,H4,R1,J1
1,0.488,0.099,0.283,0.333,0.116,0.293,S4,H4,R1,J1
2,0.107,0.008,0.204,0.150,0.032,0.159,S4,H4,R2,J2
3,0.322,0.208,0.305,0.130,0.184,0.317,S4,H4,R2,J1
4,0.242,0.280,0.131,0.210,0.191,0.351,S4,H4,R2R5,J1


In [5]:
import pandas as pd
from sklearn.cluster import KMeans
data = pd.read_excel('data.xls')
typelabel ={u'肝气郁结证型系数':'A', u'热毒蕴结证型系数':'B', u'冲任失调证型系数':'C', u'气血两虚证型系数':'D', u'脾胃虚弱证型系数':'E', u'肝肾阴虚证型系数':'F'}
keys = list(typelabel.keys())
result=pd.DataFrame()
k=4#聚类数

for i in range(len(keys)):
    kmodel=KMeans(n_clusters=k,n_jobs=-1)
    kmodel.fit(data[[keys[i]]].values)#训练模型
    r1=pd.DataFrame(kmodel.cluster_centers_,columns=[typelabel[keys[i]]])#r1是每一列对应的聚类中心
    r2 = pd.Series(kmodel.labels_).value_counts()#r2是每一类的元素数目
    r2 = pd.DataFrame(r2, columns = [typelabel[keys[i]]+'n'])#r2转化为dataframe
    r = pd.concat([r1, r2], axis = 1).sort_values(typelabel[keys[i]])#将r1和r2连接
    r.index = [1, 2, 3, 4]
    r[typelabel[keys[i]]] = r[typelabel[keys[i]]].rolling(2).mean()#相邻聚类中心的均值作为分界线，1前面没有所以结果是NAN
    r[typelabel[keys[i]]][1] = 0.0
    result=result.append(r.T)   

result.to_excel('data_processed.xls')


/home/hjz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hjz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hjz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hjz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [6]:
#自定义连接函数，用于实现L_{k-1}到C_k的连接
def connect_string(x, ms):
    x = list(map(lambda i:sorted(i.split(ms)), x))
    l = len(x[0])
    r = []
    for i in range(len(x)):
        for j in range(i,len(x)):
            if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
                r.append(x[i][:l-1]+sorted([x[j][l-1],x[i][l-1]]))
    return r
#寻找关联规则的函数
def find_rule(d, support, confidence, ms = u'--'):
    result = pd.DataFrame(index=['support', 'confidence']) #定义输出结果
  
    support_series = 1.0*d.sum()/len(d) #支持度序列
    column = list(support_series[support_series > support].index) #初步根据支持度筛选
    k = 0
  
    while len(column) > 1:
        k = k+1
        print(u'\n正在进行第%s次搜索...' %k)
        column = connect_string(column, ms)
        print(u'数目：%s...' %len(column))
        sf = lambda i: d[i].prod(axis=1, numeric_only = True) #新一批支持度的计算函数
    
        #创建连接数据
        d_2 = pd.DataFrame(list(map(sf,column)), index = [ms.join(i) for i in column]).T
    
        support_series_2 = 1.0*d_2[[ms.join(i) for i in column]].sum()/len(d) #计算连接后的支持度
        column = list(support_series_2[support_series_2 > support].index) #新一轮支持度筛选
        support_series = support_series.append(support_series_2)
        column2 = []
    
        for i in column: #遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
            i = i.split(ms)
            for j in range(len(i)):
                column2.append(i[:j]+i[j+1:]+i[j:j+1])
    
        cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
 
        for i in column2: #计算置信度序列
            cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))]/support_series[ms.join(i[:len(i)-1])]
    
        for i in cofidence_series[cofidence_series > confidence].index: #置信度筛选
            result[i] = 0.0
            result[i]['confidence'] = cofidence_series[i]
            result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
  
    result = result.T.sort_values(by=['confidence','support'], ascending = False) #结果整理，输出
    print(u'\n结果为：')
    print(result)
  
    return result

In [7]:
data = pd.read_csv('apriori.csv', header=None, dtype = object)
data.head()

,0,1,2,3,4,5,6
0,A2,B1,C3,D3,E1,F1,H1
1,A2,B1,C3,D3,E1,F1,H1
2,A2,B1,C3,D3,E1,F1,H1
3,A2,B1,C3,D3,E1,F1,H1
4,A2,B2,C3,D3,E1,F1,H1


In [8]:

data = pd.read_csv('apriori.csv', header=None, dtype = object)
print(u'\n转换原始数据至0-1矩阵...')
ct = lambda x : pd.Series(1, index = x[pd.notnull(x)]) #转换0-1矩阵的过渡函数
b = map(ct, data.values) #用map方式执行
data = pd.DataFrame(list(b)).fillna(0) #实现矩阵转换，空值用0填充
del b #删除中间变量b，节省内存

support = 0.06 #最小支持度
confidence = 0.75 #最小置信度
ms = '---' #连接符，默认'--'，用来区分不同元素，如A--B。需要保证原始表格中不含有该字符

print(u'\n开始搜索关联规则...')
find_rule(data, support, confidence, ms)



转换原始数据至0-1矩阵...

开始搜索关联规则...

正在进行第1次搜索...
数目：276...

正在进行第2次搜索...
数目：947...

正在进行第3次搜索...
数目：41...

结果为：
                    support  confidence
A3---F4---H4       0.078495    0.879518
C3---F4---H4       0.075269    0.875000
B2---F4---H4       0.062366    0.794521
C2---E3---D2       0.092473    0.754386
D2---F3---H4---A2  0.062366    0.753247


,support,confidence
A3---F4---H4,0.078495,0.879518
C3---F4---H4,0.075269,0.875000
B2---F4---H4,0.062366,0.794521
C2---E3---D2,0.092473,0.754386
D2---F3---H4---A2,0.062366,0.753247


In [9]:
data

,A2,B1,C3,D3,E1,F1,H1,B2,A1,C1,...,B4,E4,H4,D4,A4,B3,F2,C4,F3,F4
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
